In [ ]:
!pip3 install mesa

In [ ]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos un solo agente por celda elegimos `MultiGrid` que fuerza con uno o multiples objetos por celda.
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd
import random

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

In [ ]:
def get_grid(model):
  grid = np.zeros((model.grid.width, model.grid.height))
  for cell in model.grid.coord_iter():
    cell_content, x, y = cell
    for content in cell_content:
      if isinstance(content, RobotAgent):
        grid[x][y] = 6
      else:
        grid[x][y] = content.num_box
  return grid

In [ ]:
class FloorAgent(Agent):

  def __init__ (self, unique_id, model, numBoxes, stack, max_cap):
    super().__init__(unique_id, model)
    self.num_box = numBoxes
    self.priority = stack
    self.max_capacity = max_cap

In [ ]:
class RobotAgent(Agent):

  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.carryingBox = False

  def move(self):
    possible_steps = self.model.grid.get_neighborhood(
        self.pos,
        moore = False, 
        include_center = False
    )
    for neighbor in possible_steps:
      if (self.pos[0] == 0 and neighbor[0] == self.model.grid.height-1 or self.pos[0] == self.model.grid.height-1 and neighbor[0] == 0 or self.pos[1] == 0 and neighbor[1] == self.model.grid.width-1 or self.pos[1] == self.model.grid.width - 1 and neighbor[1] == 0):
        possible_steps.remove(neighbor)
    new_position = self.random.choice(possible_steps)
    robotInSpace = False

    cellmates = self.model.grid.get_cell_list_contents([new_position])
    for i in cellmates:
      if type(i) == RobotAgent:
        robotInSpace = True

    if robotInSpace == False:
      self.model.grid.move_agent(self, new_position)

  def potentialGrab(self):
    cellmates = self.model.grid.get_cell_list_contents([self.pos])
    for i in cellmates:
      if type(i) == FloorAgent:
        if i.priority == 0 and i.num_box == 1:
            self.carryingBox = True
            i.num_box -= 1
        else:
          self.move()

  def potentialRelease(self):
    cellmates = self.model.grid.get_cell_list_contents([self.pos])
    for i in cellmates:
      if type(i) == FloorAgent:
        if i.priority != 0 and i.max_capacity > i.num_box:
            i.num_box += 1
            self.carryingBox = False
            model.boxReleased()
        else:
          self.move()

  def step(self):
    if self.carryingBox == True:
      self.potentialRelease()
    elif self.carryingBox == False:
      self.potentialGrab()

In [ ]:
class StorageModel(Model):

  def __init__(self, width, height, numBoxes):
    self.grid = MultiGrid(width, height, False)
    self.schedule = SimultaneousActivation(self)
    self.num_boxes = numBoxes
    self.num_tiles = width * height
    self.placement = []
    self.totalSteps = 0

    if int(self.num_boxes) % 5 == 0:
      self.numStacks = int(self.num_boxes / 5)
      self.remainingStack = 5
    else:
      self.numStacks = int(self.num_boxes / 5) + 1
      self.remainingStack = self.num_boxes % 5

    self.boxesReleased = self.num_boxes - self.numStacks

    self.positions = list(range(0, width * height))
    self.placement = [0] * width * height

    ##Give Values to Boxes
    for i in range(self.num_boxes):
      randomNumber = np.random.choice(self.positions)
      self.placement[randomNumber] = 1
      self.positions.remove(randomNumber)

    ##Give Values to Agents
    for i in range(5):
      randomNumber = np.random.choice(self.positions)
      self.placement[randomNumber] = 6
      self.positions.remove(randomNumber)

    self.count = 0
    self.robotCount = 1

    for (content, x, y) in self.grid.coord_iter():
      if (self.placement[self.count] == 1):
        if (self.numStacks > 1):
          a = FloorAgent((x,y), self, 1, self.numStacks, 5)
          self.numStacks -= 1
        elif (self.numStacks == 1):
          a = FloorAgent((x,y), self, 1, self.numStacks, self.remainingStack)
          self.numStacks -= 1
        else:
          a = FloorAgent ((x,y), self, 1, 0, 0)
      elif (self.placement[self.count] == 0):
        a = FloorAgent((x,y), self, 0, 0, 0)
      else:
        b = FloorAgent((x,y), self, 0, 0, 0)
        self.schedule.add(b)
        self.grid.place_agent(b, (x,y))
        a = RobotAgent(self.robotCount, self)
        self.robotCount += 1
        
      self.schedule.add(a)
      self.grid.place_agent(a, (x,y))
      self.count += 1

    self.datacollector = DataCollector(model_reporters = {"Grid": get_grid})

  def boxReleased(self):
    self.boxesReleased -= 1

  def step(self):
    self.datacollector.collect(self)
    self.schedule.step()
    self.totalSteps += 1

In [ ]:
# Definimos el tamaño del Grid
GRID_SIZE_X = 10
GRID_SIZE_Y = 10

#Definimos el número de cajas
BOXES = int(input("Cantidad de cajas: "))

#Definimos el tiempo máximo de ejecución
TIEMPO_MAX = float(input("Tiempo máximo de ejecución: "))

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
model = StorageModel(GRID_SIZE_X, GRID_SIZE_Y, BOXES)
while (time.time() - start_time) < TIEMPO_MAX:
  if model.boxesReleased == 0:
    break
  model.step()

NUM_GENERATIONS = model.totalSteps

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

print("Número de movimientos realizados por todos los agentes: " + str(model.totalSteps))

Cantidad de cajas: 65
Tiempo máximo de ejecución: 15
Tiempo de ejecución: 0:00:00.633007
Número de movimientos realizados por todos los agentes: 2863


In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.binary)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=NUM_GENERATIONS)

In [ ]:
anim

Análisas:

Al acabar mi código, empezé a considerar algunas formas en la que podría hacer mi programa más eficiente en base a la cantidad de movimientos realizados por los agentes robots. En el caso de mi código actual, los agentes robots se mueven a una celda vecina aleatoria. Esto es algo que podría cambiar para disminuir el tiempo que se tarda en correr el programa. En vez de hacer que los agentes robots se muevan de forma aleatoria, lo que podría hacer sería que los agentes puedan checar las celdas de su alrededor para determinar donde hay una caja. Esto permitiría que los agentes robots puedan moverse a un lugar con caja cuando haya una a su alrededor para evitar movimientos inecesarios a lugares con celdas vacías. Otra cosa que podría implementar sería que los agentes robots tengán la ubicación de todas las stacks de cajas para que puedan moverse hacia una de ellas. Esto permitiría que los agentes robots reduzcan sus movimientos de forma drástica.